In [1]:
import grpc
import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [2]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [3]:
from keras_image_helper import create_preprocessor

In [4]:
preprocessor = create_preprocessor('xception', target_size=(299,299))

In [5]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [6]:
def np2Protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [7]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'

pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np2Protobuf(X))

In [9]:
pb_response = stub.Predict(pb_request, timeout=30.0)

In [10]:
float_preds = pb_response.outputs['dense_7'].float_val

In [11]:
classes = ['dress', 'hat', 'longsleeve',  'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']

In [12]:
dict(zip(classes, float_preds))

{'dress': -1.868290901184082,
 'hat': -4.761244773864746,
 'longsleeve': -2.316983699798584,
 'outwear': -1.062570571899414,
 'pants': 9.887160301208496,
 'shirt': -2.8124337196350098,
 'shoes': -3.6662838459014893,
 'shorts': 3.200361728668213,
 'skirt': -2.6023383140563965,
 't-shirt': -4.835045337677002}